In [1]:
import numpy as np 
import pandas as pd
import os
import datetime

In [2]:
drive_path="C:/Users/JSHARRATT/OneDrive - Department for Education/Documents/rfm_churn/"
path=os.path.join(drive_path, "scanner_data.csv")

In [3]:
data=pd.read_csv(path)
data.Date=pd.to_datetime(data.Date)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      131706 non-null  int64         
 1   Date            131706 non-null  datetime64[ns]
 2   Customer_ID     131706 non-null  int64         
 3   Transaction_ID  131706 non-null  int64         
 4   SKU_Category    131706 non-null  object        
 5   SKU             131706 non-null  object        
 6   Quantity        131706 non-null  float64       
 7   Sales_Amount    131706 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 8.0+ MB


C:\Users\JSHARRATT\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/01/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\JSHARRATT\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/01/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\JSHARRATT\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/01/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\JSHARRATT\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/01/2016' in DD/MM/YYYY format. Provide 

In [4]:
df=data[["Date", "Customer_ID", "Sales_Amount"]]

In [5]:
len(df)
df.dtypes

Date            datetime64[ns]
Customer_ID              int64
Sales_Amount           float64
dtype: object

In [13]:
cutoff=datetime.date(2016,6,1)
cutoff=pd.to_datetime(cutoff)


In [31]:
def get_frequency(data, cutoff, date_column, customer_column, sale_column, freq='M'):
    observed=data[data[date_column]<cutoff].copy()
    observed.set_index("Date", inplace=True) #set index sets index using date column in this context
    observed.index=pd.DatetimeIndex(observed.index)
    observed=observed.groupby(
    [customer_column, pd.Grouper(freq=freq, level=date_column)]).count()
    observed=observed.rename(columns={sale_column:"Value"})
    observed.Value=1
    return observed.groupby(customer_column).sum().reset_index().rename(columns={"Value":"Frequency"})

def get_recency(data, cutoff, date_column, customer_column, sale_column, freq="M"):
    recency=data[data[date_column]<cutoff].copy()
    recency=recency.groupby(customer_column)[date_column].max()
    return (cutoff-recency).reset_index().rename(columns={date_column:'Recency'})

def get_value(data, cutoff, date_column, customer_column, sale_column, freq="M" ):
    value=data[data[date_column]<cutoff].copy()
    return value.groupby(customer_column)[sale_column].mean().reset_index().rename(columns={sale_column:"Value"})

def get_age(data, cutoff, date_column, customer_column, sale_column, freq="M"):
    age=data[data[date_column]<cutoff].copy()
    age=age.groupby(customer_column)[date_column].min()
    return (cutoff-age).reset_index().rename(columns={date_column:'Age'})

def get_rfm(data, cutoff, date_column, customer_column, sale_column, freq="M"):
    frequency=get_frequency(data, cutoff, date_column, customer_column, sale_column, freq='M')
    recency=get_recency(data, cutoff, date_column, customer_column, sale_column, freq='M')
    value=get_value(data, cutoff, date_column, customer_column, sale_column, freq='M')
    age=get_age(data, cutoff, date_column, customer_column, sale_column, freq='M')
    return frequency.merge(recency, on=customer_column).merge(value, on=customer_column).merge(age, on=customer_column)



    
    
    
    

In [34]:
df_featured=get_rfm(df, cutoff, "Date", "Customer_ID", "Sales_Amount", freq='M')

In [46]:
#pd.to_numeric(df_featured.Age)
df_featured.Age=df_featured.Age.dt.days
df_featured.Recency=df_featured.Recency.dt.days


0        131
1         69
2        121
3        100
4         62
        ... 
13662     20
13663     50
13664    142
13665    110
13666    110
Name: Age, Length: 13667, dtype: int64

In [47]:
df_featured.head()

,Customer_ID,Frequency,Recency,Value,Age
0,1,1,131,8.145000,131
1,2,1,69,7.770000,69
2,3,1,121,3.640000,121
3,5,3,4,14.672500,100
4,6,1,62,8.516667,62


In [1]:
print("Hi")

Hi
